## CNN Architecture

### AlexNet (modified)

In [1]:
import torch

train_on_gpu = torch.cuda.is_available()

# check if CUDA is available
device = torch.device("cuda:0" if train_on_gpu else "cpu")
print(device)
if not train_on_gpu:
    print('Training on CPU ...')
else:
    print('Training on GPU ...')

cuda:0
Training on GPU ...


In [2]:
import torch.nn as nn
import torch.nn.functional as F

class AlexNet(nn.Module):

    def __init__(self, num_classes=4):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv1d(22, 64, kernel_size=22, stride=2),
            nn.BatchNorm1d(64, affine=False),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool1d(kernel_size=12, stride=2),
            nn.Dropout(p=0.8),
            nn.Conv1d(64, 192, kernel_size=12),
            nn.BatchNorm1d(192, affine=False),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3, stride=2),
            nn.Dropout(p=0.8),
            nn.Conv1d(192, 384, kernel_size=4, stride=2),
            nn.BatchNorm1d(384, affine=False),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(p=0.8),
            nn.Conv1d(384, 256, kernel_size=4, stride=2),
            nn.BatchNorm1d(256, affine=False),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(p=0.8),
            nn.Conv1d(256, 256, kernel_size=4),
            nn.BatchNorm1d(256, affine=False),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool1d(kernel_size=4, stride=1),
            
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.BatchNorm1d(256 * 21),
            nn.LeakyReLU(inplace=True),
            nn.BatchNorm1d(256 * 21),
            nn.LeakyReLU(inplace=True),
            nn.Linear(256 * 21, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# create a complete CNN
model1 = AlexNet()
model2 = AlexNet()
#model3 = AlexNet()

# move tensors to GPU if CUDA is available
if train_on_gpu:
    model1.cuda()
    model2.cuda()
    #model3.cuda()

### Load the Model with the Lowest Validation Loss

In [3]:
model1.load_state_dict(torch.load('model_EEG_67%.pt'))
model2.load_state_dict(torch.load('model_EEG_65%.pt'))
#model3.load_state_dict(torch.load('model_EEG_60%.pt'))

<All keys matched successfully>

### Test the Trained Network

In [4]:
import numpy as np
from torch.utils import data

batch_size = 200
criterion = nn.CrossEntropyLoss()

X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
y_test -= 769
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).long()
test_data = data.TensorDataset(X_test, y_test)

test_loader = data.DataLoader(test_data, batch_size=batch_size)

In [5]:
# specify the target classes
classes = [769, 770, 771, 772]

# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(4))
class_total = list(0. for i in range(4))

model1.eval()
model2.eval()
#model3.eval()
for data, target in test_loader:
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    output1 = model1(data)
    output2 = model2(data)
    #output3 = model3(data)
    #output = (output1 + output2 + output3)/3
    output = (output1 + output2)/2
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    #print(pred)
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(correct.shape[0]):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1
# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(4):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.863625

Test Accuracy of   769: 76% (85/111)
Test Accuracy of   770: 74% (95/127)
Test Accuracy of   771: 65% (63/96)
Test Accuracy of   772: 59% (65/109)

Test Accuracy (Overall): 69% (308/443)
